<a href="https://colab.research.google.com/github/allenyeh929/generative_ai/blob/main/hw7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 下載資料庫、安裝套件

In [ ]:
!wget -O faiss_db.zip 'https://drive.google.com/uc?export=download&id=1-fKq0ro1-iihtJHKCwwCjLxf9uKmn1pM'
!unzip -o faiss_db.zip
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

## 2. 下載資料庫、安裝套件

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

from openai import OpenAI
import gradio as gr
import os

# E5 Embedding 包裝
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

# 向量資料庫 + 查詢系統
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()


## 3. 設定模型連線與 Prompt

In [ ]:
from google.colab import userdata
api_key = userdata.get('Groq')  # 載入 Groq API 金鑰
os.environ["OPENAI_API_KEY"] = api_key

model = "llama3-70b-8192"  # 改為你用的 Groq 模型名稱
base_url = "https://api.groq.com/openai/v1"
client = OpenAI(base_url=base_url)

# ✅ System Prompt：固定用繁體中文、專業但易懂地回答電腦圖學問題
system_prompt = '''
請務必全程使用「繁體中文」來回答所有問題，不論使用者使用什麼語言提問，回答時都只能使用繁體中文。這一點非常重要，請嚴格遵守，不得違反。

你是一個精通電腦圖學的 AI 系統，擅長解釋電腦圖學論文中的技術概念。請以專業、邏輯清楚、淺顯易懂且詳細的方式回答。

不可以使用英文、簡體中文或其他語言，所有回答都必須是繁體中文，否則將視為錯誤回應。
'''

# ✅ Prompt Template：插入取回文件與使用者問題
prompt_template = """
根據下列電腦圖學論文資料回覆使用者問題：

{retrieved_chunks}

使用者的問題是：{question}

請用繁體中文，並以邏輯清楚、淺顯易懂的方式詳細回答。
若回答使用英文，則代表錯誤。
"""


## 4. 建立對話處理函式

In [ ]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history
    docs = retriever.invoke(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": final_prompt}
        ]
    )

    answer = response.choices[0].message.content
    chat_history.append((user_input, answer))
    return answer


## 5. Gradio 介面

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 📘 電腦圖學問答助手")
    gr.Markdown("""
歡迎使用 **電腦圖學問答助手**，這是一個透過 RAG 技術建構的互動系統，專門回答有關 **電腦圖學論文、技術、概念與方法** 的問題。

✅ 特點：
- 回答使用繁體中文
- 精通圖形學論文與概念（如：光線追蹤、著色器、渲染、圖形硬體等）
- 可幫助理解技術名詞、核心思想與應用情境

請輸入你對電腦圖學的疑問，我會幫你查找資料並提供清楚的說明。
""")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)
